# Tutorial of VQE
In this example, we will demonstrate the variational algorithms, *variational quantum eigensolver* (VQE), using the sampling estimator of this challenge. The code of VQE is based on the the quri-parts tutorial.

In [1]:
import sys
sys.path.append("../")

import numpy as np

from quri_parts.algo.ansatz import HardwareEfficient
from quri_parts.algo.optimizer import OptimizerStatus, Adam
from quri_parts.core.estimator.gradient import parameter_shift_gradient_estimates
from quri_parts.core.state import ParametricCircuitQuantumState
from quri_parts.core.measurement import bitwise_commuting_pauli_measurement
from quri_parts.core.sampling.shots_allocator import (
    create_equipartition_shots_allocator,
)
from quri_parts.core.operator import Operator, pauli_label, PAULI_IDENTITY
from utils.challenge_2023 import ChallengeSampling, QuantumCircuitTimeExceededError

## Setup

In [2]:
n_qubits = 4

# define an operator to be applied VQE 
hamiltonian = Operator({
    pauli_label("Z0"): 0.25,
    pauli_label("Z1 Z2"): 2.0,
    pauli_label("X1 X2"): 0.5 + 0.25j,
    pauli_label("Z1 Y3"): 1.0j,
    pauli_label("Z2 Y3"): 1.5 + 0.5j,
    pauli_label("X1 Y3"): 2.0j,
    PAULI_IDENTITY: 3.0,
})

# define an ansatz

hw_ansatz = HardwareEfficient(qubit_count=n_qubits, reps=1)

# make a parametric state
parametric_state = ParametricCircuitQuantumState(n_qubits, hw_ansatz)

In order to use the sampling_estimator, we need to define some inputs. See the sampling tutorial. 

In [3]:
hardware_type = "sc"
shots_allocator = create_equipartition_shots_allocator()
measurement_factory = bitwise_commuting_pauli_measurement
n_shots = 10**5

Define a sampling estimator by using the above inputs. In this tutorial, we use the noiseless sampling_estimator.

In [4]:
challenge_sampling = ChallengeSampling(noise=False)
sampling_estimator = challenge_sampling.create_concurrent_parametric_sampling_estimator(
                        n_shots,
                        measurement_factory,
                        shots_allocator,
                        hardware_type
                    )

# Define some functions to construct VQE
In this challenge, there is a limitation to the circuit run-time. When the sampling-estimator (even in the sampler) reach the limit, they output an error `QuantumCircuitTimeExceededError` . So when the error is output, you can get the previous result by using `TryExcept`.

In [5]:
# define an optimizer
adam_optimizer = Adam(ftol=10e-5)

# define a cost function

def cost_fn(hamiltonian, parametric_state, param_values, estimator):
    estimate = estimator(hamiltonian, parametric_state, [param_values])
    return estimate[0].value.real

def vqe(hamiltonian, parametric_state, estimator, init_params, optimizer):
    opt_state = optimizer.get_init_state(init_params)

    def c_fn(param_values):
        return cost_fn(hamiltonian, parametric_state, param_values, estimator)

    def g_fn(param_values):
        grad = parameter_shift_gradient_estimates(hamiltonian,parametric_state, param_values, estimator)
        # In this tutorial, we use the parameter shift rule to calculate the gradient
        return np.asarray([i.real for i in grad.values])

    while True:
        try:
            opt_state = optimizer.step(opt_state, c_fn, g_fn)
        except QuantumCircuitTimeExceededError:
            print("Reached the limit of shots")
            return opt_state

        if opt_state.status == OptimizerStatus.FAILED:
            print("Optimizer failed")
            break
        if opt_state.status == OptimizerStatus.CONVERGED:
            print("Optimizer converged")
            break
    return opt_state

## Run VQE

In [6]:
# set initial parameters
init_param = [i for i in range(hw_ansatz.parameter_count)]

# vqe part
result = vqe(hamiltonian, parametric_state, sampling_estimator, init_param, adam_optimizer)

# exact value
exact_value = -1.8696938456699062

print("n_iter", result.niter)
print("vqe_energy", result.cost)
print("exact energy:", exact_value)
challenge_sampling.reset()

Optimizer converged
n_iter 255
vqe_energy -1.311961119611196
exact energy: -1.8696938456699062
